<a href="https://colab.research.google.com/github/angelelliott/miso-bert/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
transformers.__version__

'4.24.0'

In [ ]:
#lowercase, change urls for url

train = pd.read_excel("/content/drive/MyDrive/Tests/es_AMI_TrainingSet.xlsx")
test = pd.read_csv("/content/drive/MyDrive/Tests/es_AMI_TestSet_labelled_DEBIASED")
test = test.sample(n=len(test))
import re
url = re.compile(r'http\S+')
#mention = re.compile(r'@\S+')
train['tweet'] =  train['tweet'].str.lower()
train['tweet'] = [url.sub('url', x) for x in train['tweet']]
#train['tweet']= [mention.sub('usuario', x) for x in train['tweet']]
test['tweet'] =  test['tweet'].str.lower()
test['tweet'] = [url.sub('url', x) for x in test['tweet']]
#test['tweet']= [mention.sub('usuario', x) for x in test['tweet']]

In [ ]:
!pip install emoji --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import emoji

In [ ]:
train['tweet'] = [emoji.demojize(x, language='es') for x in train['tweet']]
test['tweet'] = [emoji.demojize(x, language='es') for x in test['tweet']]

In [ ]:
test_texts = test[["tweet"]]
test_labels = test["misogynous"].ravel()

train_X = train[['tweet']]
train_y = train['misogynous'].ravel()

In [ ]:
# split training set into train and validation
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_X, train_y, test_size=.2)

In [ ]:
# tokenize text

# Import pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/myModels/pretrained_model_concat3epc")
tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/myModels/pretrained_model_concat3epc")

Some weights of the model checkpoint at /content/drive/MyDrive/myModels/pretrained_model_concat3epc were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/My

In [ ]:
train['Avg_length'] = train["tweet"].apply(lambda x: len(x.split())/len(x.split('.')))
train['Avg_length'].describe()

count    3307.000000
mean       13.549541
std         9.210529
min         0.750000
25%         7.000000
50%        11.000000
75%        18.000000
max        59.000000
Name: Avg_length, dtype: float64

In [ ]:
train_encodings = tokenizer(train_texts['tweet'].tolist(), return_tensors='pt', max_length=80, padding=True)
val_encodings = tokenizer(val_texts['tweet'].tolist(), return_tensors='pt', max_length=80, padding=True)
test_encodings = tokenizer(test_texts['tweet'].tolist(), return_tensors='pt', max_length=80, padding=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2323: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "


In [ ]:
class MisogynyDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

In [ ]:
train_dataset = MisogynyDataset(train_encodings, train_labels)
val_dataset = MisogynyDataset(val_encodings, val_labels)
test_dataset = MisogynyDataset(test_encodings, test_labels)

In [ ]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total # of training epochs
    per_device_train_batch_size=16, # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.00001,               # strength of weight decay
    learning_rate = 5e-5,
    logging_dir='./logs',            # directory for storing logs
    optim = 'adamw_torch'
)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 2645
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 332
  Number of trainable parameters = 109852418
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=332, training_loss=0.4140738521713808, metrics={'train_runtime': 147.993, 'train_samples_per_second': 35.745, 'train_steps_per_second': 2.243, 'total_flos': 445829349976800.0, 'train_loss': 0.4140738521713808, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 662
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


{'eval_loss': 0.4147123694419861,
 'eval_runtime': 7.2862,
 'eval_samples_per_second': 90.857,
 'eval_steps_per_second': 2.882,
 'epoch': 2.0}

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
predictions = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 831
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


In [ ]:
compute_metrics(predictions)

{'accuracy': 0.8459687123947052,
 'f1': 0.830238726790451,
 'precision': 0.7964376590330788,
 'recall': 0.8670360110803325}

In [ ]:
predictions.predictions.argmax(-1)

array([1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,

In [ ]:
predictions.label_ids

array([0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,

In [ ]:
tweetsListform = test_texts['tweet'].tolist()

for i in range(len(tweetsListform)):
  if predictions.predictions.argmax(-1)[i] != predictions.label_ids[i]:
    print("Tweet:", tweetsListform [i])
    print("Correct label:", predictions.label_ids[i])
    print("Assigned label: ", predictions.predictions.argmax(-1)[i])


Tweet: luego en marzo que adán saque la bufanda rosa del día de la mujer. hoy dice 'mi novia es sevillista y es una puta más'. ojalá sea un fake aunque no lo parece. url
Correct label: 0
Assigned label:  1
Tweet: mi prima está esta mirando el partido y le dije q iban a perder y me dice cállate hija de puta q amor
Correct label: 0
Assigned label:  1
Tweet: @pasivosumiso9 mi polla en tu culo!
Correct label: 0
Assigned label:  1
Tweet: @julia55445365 julia vos tambien sos hermosa la puta madre callate
Correct label: 0
Assigned label:  1
Tweet: @hdeftz @spainonymous @dieantfood @lionconk sería para partirse la polla si ahora yo pongo que todas las mujeres son basura.
Correct label: 1
Assigned label:  0
Tweet: adan, portero del betis no solo es tonto sino que lo celebra al cantico de ' mi novia es sevillista, una puta mas', tampoco se le puede pedir mas al chaval
Correct label: 0
Assigned label:  1
Tweet: @juantaverash el acoso en la mujer es tan solo cuando el hombre no le gusta, si el hom